In [1]:
import os 
os.chdir('../')

In [2]:
import orca
from collections import OrderedDict

from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
from urbansim_templates.utils import get_data, get_df

from choicemodels.tools import MergedChoiceTable

from urbansim.models.util import columns_in_formula, apply_filter_query

In [3]:
region_code = '06197001'
orca.add_injectable('running_calibration_routine', False)
orca.add_injectable('local_simulation', True)
orca.add_injectable('initial_run', False)
orca.add_injectable('region_code', region_code)
orca.add_injectable('base_year', 2010)
orca.add_injectable('forecast_year', 2011)
orca.add_injectable('calibrated', True)
orca.add_injectable('calibrated_folder', 'custom')
orca.add_injectable('multi_level_lcms', True)
orca.add_injectable('segmented_lcms', True)
orca.add_injectable('capacity_boost', 1)
orca.add_injectable('all_local', True)
orca.add_injectable('skim_source', 'beam')

In [4]:
import datasources
import variables
import models

importing datasources
importing datasources for region 06197001
custom_mpo_06197001_model_data.h5
Checking if custom_settings.yaml file exists
Checking if custom output_parameters.yaml file exists


/Users/juandavidcaicedocastro/Dropbox/01_berkeley/22_UrbanSim/01_projects/MLCM/02_github/DEMOS_URBANSIM/demos_urbansim/datasources.py:357: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hct = hct.append(forecast_hct.reset_index())
/Users/juandavidcaicedocastro/Dropbox/01_berkeley/22_UrbanSim/01_projects/MLCM/02_github/DEMOS_URBANSIM/demos_urbansim/datasources.py:389: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ect = ect.append(forecast_ect.reset_index())


Output path exists!
importing variables for region 06197001
importing models for region 06197001
Checking if value configs exist
Checking if rent configs exist


In [5]:
configs_folder = 'configs/calibrated_configs/custom/06197001'
mm.initialize(configs_folder)
orca.run(orca.get_injectable('pre_processing_steps'))

Registering model step 'hlcm_county_own_1p_54less_pf'
Registering model step 'rdplcm_06081_blocks_pf'
Registering model step 'hlcm_06013_blocks_rent_1p_54less_pf'
Registering model step 'elcm_06097_blocks_1_pf'
Registering model step 'hlcm_06001_blocks_pf'
Registering model step 'mortality'
Registering model step 'hlcm_06075_blocks_own_2p_54less_pf'
Registering model step 'elcm_06001_blocks_2_pf'
Registering model step 'elcm_06041_blocks_3_pf'
Registering model step 'hlcm_06081_blocks_own_1p_54less_pf'
Registering model step 'elcm_06075_blocks_0_pf'
Registering model step 'hlcm_06013_blocks_pf'
Registering model step 'rdplcm_06013_blocks_sf_pf'
Registering model step 'hlcm_06055_blocks_own_1p_54less_pf'
Registering model step 'hlcm_06013_blocks_own_2p_54less_pf'
Registering model step 'hlcm_06085_blocks_own_2p_54less_pf'
Registering model step 'elcm_06013_blocks_4_pf'
Registering model step 'hlcm_06085_blocks_rent_2p_54less_pf'
Registering model step 'hlcm_06097_blocks_own_1p_55plus_pf

Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 92395
Setting CH edge vector of size 232266
Range graph removed 233224 edges of 464532
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
Precomputing network for distance 1000.
Network precompute starting.
Network precompute done.
Time to execute step 'build_networks': 4.72 s
Running step 'generate_outputs'
Generating outputs for (year 2010, forecast year 2011)...
Time to execute step 'generate_outputs': 0.00 s
Total time to execute iteration 1 with iteration value None: 4.72 s


/Users/juandavidcaicedocastro/Dropbox/01_berkeley/22_UrbanSim/01_projects/MLCM/02_github/DEMOS_URBANSIM/demos_urbansim/variables.py:1406: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  coords = blocks.local.groupby('block_group_id').mean().reset_index()
/Users/juandavidcaicedocastro/Dropbox/01_berkeley/22_UrbanSim/01_projects/MLCM/02_github/DEMOS_URBANSIM/demos_urbansim/variables.py:1414: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  coords = blocks.local.groupby('block_group_id').mean().reset_index()


In [6]:
county_ids = orca.get_table('blocks').county_id.unique()

In [7]:
travel_data = orca.get_table('travel_data').to_frame(columns = ['logsum'])

In [8]:
# orca.get_table('travel_data').local

In [59]:
travel_time_col = 'logsum'
mandatory_decision_rule = 'min'

mct_intx_ops_min = OrderedDict({
    'extra_alts_cols': ['zone_id'],
    'successive_merges': [{
        'right_table': 'persons',
        'right_cols': [
            'household_id', 'mandatory_work_zone_id', 'mandatory_school_zone_id',
            'mandatory_school_dummy', 'mandatory_work_dummy', 'mandatory_activity_dummy'],
        'mct_cols': ['household_id', 'zone_id'],
        'on_cols': ['household_id']
    }, {
        'right_table': 'travel_data',
        'left_on': ['zone_id', 'mandatory_work_zone_id'],
        'right_index': True,
        'how': 'left'
    }, {
        'right_table': 'travel_data',
        'left_on': ['zone_id', 'mandatory_school_zone_id'],
        'right_index': True,
        'how': 'left',
        'suffixes': ['_work', '_school']
        }],
    'aggregations': {
        '{0}_work'.format(travel_time_col): mandatory_decision_rule,
        '{0}_school'.format(travel_time_col): mandatory_decision_rule,
        'mandatory_work_dummy': 'max',
        'mandatory_school_dummy': 'max',
        'mandatory_activity_dummy': 'max'},
    'rename_cols': {
        '{0}_work'.format(travel_time_col): f'{mandatory_decision_rule}_work_{travel_time_col}',
        '{0}_school'.format(travel_time_col): f'{mandatory_decision_rule}_school_{travel_time_col}'},
    'sequential_eval_ops': [{
        'name': 'no_work',
        'expr': '1 - mandatory_work_dummy'
    }, {
        'name': 'work_nan_fill',
        'expr': f'{mandatory_decision_rule}_work_logsum.fillna(0)',
        'engine': 'python'
    }, {
        'name': 'school_nan_fill',
        'expr': f'{mandatory_decision_rule}_school_logsum.fillna(0)',
        'engine': 'python'
    }, {
        'name': f'mandatory_{travel_time_col}',
        'expr': '(work_nan_fill * mandatory_work_dummy) + (no_work * school_nan_fill * mandatory_school_dummy)'
    }]
})

In [60]:
travel_time_col = 'logsum'
mandatory_decision_rule = 'max'

mct_intx_ops_max = OrderedDict({
    'extra_alts_cols': ['zone_id'],
    'successive_merges': [{
        'right_table': 'persons',
        'right_cols': [
            'household_id', 'mandatory_work_zone_id', 'mandatory_school_zone_id',
            'mandatory_school_dummy', 'mandatory_work_dummy', 'mandatory_activity_dummy'],
        'mct_cols': ['household_id', 'zone_id'],
        'on_cols': ['household_id']
    }, {
        'right_table': 'travel_data',
        'left_on': ['zone_id', 'mandatory_work_zone_id'],
        'right_index': True,
        'how': 'left'
    }, {
        'right_table': 'travel_data',
        'left_on': ['zone_id', 'mandatory_school_zone_id'],
        'right_index': True,
        'how': 'left',
        'suffixes': ['_work', '_school']
        }],
    'aggregations': {
        '{0}_work'.format(travel_time_col): mandatory_decision_rule,
        '{0}_school'.format(travel_time_col): mandatory_decision_rule,
        'mandatory_work_dummy': 'max',
        'mandatory_school_dummy': 'max',
        'mandatory_activity_dummy': 'max'},
    'rename_cols': {
        '{0}_work'.format(travel_time_col): f'{mandatory_decision_rule}_work_{travel_time_col}',
        '{0}_school'.format(travel_time_col): f'{mandatory_decision_rule}_school_{travel_time_col}'},
    'sequential_eval_ops': [{
        'name': 'no_work',
        'expr': '1 - mandatory_work_dummy'
    }, {
        'name': 'work_nan_fill',
        'expr': f'{mandatory_decision_rule}_work_logsum.fillna(0)',
        'engine': 'python'
    }, {
        'name': 'school_nan_fill',
        'expr': f'{mandatory_decision_rule}_school_logsum.fillna(0)',
        'engine': 'python'
    }, {
        'name': f'mandatory_{travel_time_col}',
        'expr': '(work_nan_fill * mandatory_work_dummy) + (no_work * school_nan_fill * mandatory_school_dummy)'
    }]
})

In [61]:
hlcm_segments = [
    'own_1p_54less', 'own_1p_55plus', 'own_2p_54less', 'own_2p_55plus', 'rent_1p_54less',
    'rent_1p_55plus', 'rent_2p_54less', 'rent_2p_55plus']

In [64]:
for segment in hlcm_segments:
    for county_id in county_ids:
        model_name = 'hlcm_%s_blocks_%s_pf' % (county_id, segment)
        model_old = mm.get_step(model_name)
        model = mm.get_step(model_name)
        
        model.alt_filters = [
            filter.strip() for filter in model.out_alt_filters.split('&') if 'county_id' in filter]
            
            
        alternatives = get_data(
            model.alternatives,
            model_expression=model.model_expression,
            filters=model.alt_filters,
            extra_columns=['zone_id']
        )
        
        model.out_alt_filters += ' & (zone_id == zone_id)'
        
        model.chooser_filters = [
            filter for filter in model.out_chooser_filters.split('&') if 'block_id' not in filter]
        choosers = get_data(
            model.choosers,
            filters=model.chooser_filters,
            model_expression=model.model_expression,
            extra_columns=[model.choice_column]
        )
        
        
        
        # Max Impedance
        
        mct = MergedChoiceTable(
            choosers,
            alternatives,
            chosen_alternatives=model.choice_column,
            sample_size=model.alt_sample_size
        )
        model.mct_intx_ops = mct_intx_ops_max
        new_mct_max = model.perform_mct_intx_ops(mct)
        
        model.model_expression = 'mandatory_activity_dummy:mandatory_logsum + ' + model_old.model_expression[41:]
        print("\MAX MODEL SPEC")
        model.fit(new_mct_max)
        mm.register(model)
        
        # Min Impedance
        
        mct = MergedChoiceTable(
            choosers,
            alternatives,
            chosen_alternatives=model.choice_column,
            sample_size=model.alt_sample_size
        )
        model.mct_intx_ops = mct_intx_ops_min
        new_mct_min = model.perform_mct_intx_ops(mct)
        
        model.model_expression = 'mandatory_activity_dummy:mandatory_logsum + ' + model_old.model_expression[41:]
        print("\nMIN MODEL SPEC")
        model.fit(new_mct_min)
        
        #No Impedance
        mct = MergedChoiceTable(
            choosers,
            alternatives,
            chosen_alternatives=model.choice_column,
            sample_size=model.alt_sample_size
        )
        model.model_expression = model_old.model_expression[41:]
        print("\No Logsum MODEL SPEC")
        model.fit(mct)

Replacing MCT None's and NaN's with 0
\MAX MODEL SPEC
                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:          6,241
Model:         Multinomial Logit   Df Residuals:              6,216
Method:       Maximum Likelihood   Df Model:                     25
Date:                 2023-06-29   Pseudo R-squ.:             0.119
Time:                      17:38   Pseudo R-bar-squ.:         0.118
AIC:                  50,685.677   Log-Likelihood:      -25,317.839
BIC:                  50,854.150   LL-Null:             -28,740.867
                                               coef   std err         z     P>|z|   Conf. Int.
----------------------------------------------------------------------------------------------
mandatory_activity_dummy:mandatory_logsum    2.7752     0.014   200.251     0.000             
st_bg_mean_age_of_head                       0.3092     0.023    13.649     0.000             
st_bg_mean_children   

Replacing MCT None's and NaN's with 0
\MAX MODEL SPEC
                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:          4,257
Model:         Multinomial Logit   Df Residuals:              4,232
Method:       Maximum Likelihood   Df Model:                     25
Date:                 2023-06-29   Pseudo R-squ.:             0.106
Time:                      17:39   Pseudo R-bar-squ.:         0.105
AIC:                  35,109.816   Log-Likelihood:      -17,529.908
BIC:                  35,268.724   LL-Null:             -19,604.209
                                               coef   std err         z     P>|z|   Conf. Int.
----------------------------------------------------------------------------------------------
mandatory_activity_dummy:mandatory_logsum    2.7255     0.018   151.488     0.000             
st_bg_mean_age_of_head                       0.3550     0.030    11.861     0.000             
st_bg_mean_children   

Replacing MCT None's and NaN's with 0
\MAX MODEL SPEC
                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:          1,419
Model:         Multinomial Logit   Df Residuals:              1,394
Method:       Maximum Likelihood   Df Model:                     25
Date:                 2023-06-29   Pseudo R-squ.:             0.084
Time:                      17:40   Pseudo R-bar-squ.:         0.080
AIC:                  12,025.740   Log-Likelihood:       -5,987.870
BIC:                  12,157.183   LL-Null:              -6,534.736
                                               coef   std err         z     P>|z|   Conf. Int.
----------------------------------------------------------------------------------------------
mandatory_activity_dummy:mandatory_logsum    2.0742     0.035    59.527     0.000             
st_bg_mean_age_of_head                       0.3944     0.057     6.891     0.000             
st_bg_mean_children   

Replacing MCT None's and NaN's with 0
\MAX MODEL SPEC
                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:            517
Model:         Multinomial Logit   Df Residuals:                492
Method:       Maximum Likelihood   Df Model:                     25
Date:                 2023-06-29   Pseudo R-squ.:             0.080
Time:                      17:41   Pseudo R-bar-squ.:         0.069
AIC:                   4,431.760   Log-Likelihood:       -2,190.880
BIC:                   4,537.961   LL-Null:              -2,380.873
                                               coef   std err         z     P>|z|   Conf. Int.
----------------------------------------------------------------------------------------------
mandatory_activity_dummy:mandatory_logsum    2.1113     0.060    35.282     0.000             
st_bg_mean_age_of_head                       0.4846     0.101     4.775     0.000             
st_bg_mean_children   

Replacing MCT None's and NaN's with 0
\MAX MODEL SPEC
                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:          7,186
Model:         Multinomial Logit   Df Residuals:              7,161
Method:       Maximum Likelihood   Df Model:                     25
Date:                 2023-06-29   Pseudo R-squ.:             0.144
Time:                      17:42   Pseudo R-bar-squ.:         0.143
AIC:                  56,727.040   Log-Likelihood:      -28,338.520
BIC:                  56,899.037   LL-Null:             -33,092.753
                                               coef   std err         z     P>|z|   Conf. Int.
----------------------------------------------------------------------------------------------
mandatory_activity_dummy:mandatory_logsum    2.2491     0.015   150.919     0.000             
st_bg_mean_age_of_head                       0.0022     0.027     0.084     0.933             
st_bg_mean_children   

...


MIN MODEL SPEC
                  CHOICEMODELS ESTIMATION RESULTS                   
Dep. Var.:                chosen   No. Observations:          22,103
Model:         Multinomial Logit   Df Residuals:              22,079
Method:       Maximum Likelihood   Df Model:                      24
Date:                 2023-06-29   Pseudo R-squ.:              0.154
Time:                      20:58   Pseudo R-bar-squ.:          0.154
AIC:                 172,271.437   Log-Likelihood:       -86,111.718
BIC:                 172,463.520   LL-Null:             -101,788.077
                                          coef   std err         z     P>|z|   Conf. Int.
-----------------------------------------------------------------------------------------
st_bg_mean_age_of_head                  0.4244     0.011    37.671     0.000             
st_bg_mean_children                     0.1310     0.013     9.832     0.000             
st_bg_mean_year_built                  -0.1083     0.011    -9.679     0


MIN MODEL SPEC
                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:          8,057
Model:         Multinomial Logit   Df Residuals:              8,033
Method:       Maximum Likelihood   Df Model:                     24
Date:                 2023-06-29   Pseudo R-squ.:             0.187
Time:                      21:00   Pseudo R-bar-squ.:         0.186
AIC:                  60,393.918   Log-Likelihood:      -30,172.959
BIC:                  60,561.781   LL-Null:             -37,103.856
                                          coef   std err         z     P>|z|   Conf. Int.
-----------------------------------------------------------------------------------------
st_bg_mean_age_of_head                  0.6660     0.024    27.666     0.000             
st_bg_mean_children                    -0.0237     0.019    -1.230     0.219             
st_bg_mean_year_built                   0.0312     0.028     1.114     0.265    


MIN MODEL SPEC
                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:         11,141
Model:         Multinomial Logit   Df Residuals:             11,117
Method:       Maximum Likelihood   Df Model:                     24
Date:                 2023-06-29   Pseudo R-squ.:             0.170
Time:                      21:02   Pseudo R-bar-squ.:         0.170
AIC:                  85,194.933   Log-Likelihood:      -42,573.466
BIC:                  85,370.574   LL-Null:             -51,306.201
                                          coef   std err         z     P>|z|   Conf. Int.
-----------------------------------------------------------------------------------------
st_bg_mean_age_of_head                  0.5483     0.017    31.705     0.000             
st_bg_mean_children                     0.0744     0.013     5.790     0.000             
st_bg_mean_year_built                  -0.1027     0.020    -5.161     0.000    

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:          1,326
Model:         Multinomial Logit   Df Residuals:              1,302
Method:       Maximum Likelihood   Df Model:                     24
Date:                 2023-06-29   Pseudo R-squ.:             0.241
Time:                      21:03   Pseudo R-bar-squ.:         0.237
AIC:                   9,314.017   Log-Likelihood:       -4,633.008
BIC:                   9,438.575   LL-Null:              -6,106.456
                                          coef   std err         z     P>|z|   Conf. Int.
-----------------------------------------------------------------------------------------
st_bg_mean_age_of_head                  0.6244     0.046    13.479     0.000             
st_bg_mean_children                     0.1438     0.042     3.457     0.001             
st_bg_mean_year_built                  -0.0282     0.057    -0.493     0.622             
st_bg_

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:          1,223
Model:         Multinomial Logit   Df Residuals:              1,199
Method:       Maximum Likelihood   Df Model:                     24
Date:                 2023-06-29   Pseudo R-squ.:             0.167
Time:                      21:03   Pseudo R-bar-squ.:         0.163
AIC:                   9,428.714   Log-Likelihood:       -4,690.357
BIC:                   9,551.332   LL-Null:              -5,632.123
                                          coef   std err         z     P>|z|   Conf. Int.
-----------------------------------------------------------------------------------------
st_bg_mean_age_of_head                  0.6290     0.047    13.365     0.000             
st_bg_mean_children                     0.1665     0.046     3.631     0.000             
st_bg_mean_year_built                   0.0017     0.064     0.027     0.978             
st_bg_

In [17]:
segment = hlcm_segments[0]
county_id = county_ids[0]

In [18]:
model_name = 'hlcm_%s_blocks_%s_pf' % (county_id, segment)
model_old = mm.get_step(model_name)
model = mm.get_step(model_name)

In [19]:
model.filters = [filter.strip() for filter in model.out_alt_filters.split('&') if 'county_id' in filter]

In [20]:
alternatives = get_data(
                model.alternatives,
                model_expression=model.model_expression,
                filters=model.alt_filters,
                extra_columns=['zone_id']
            )

In [26]:
alternatives.head()

,zone_id,county_id,st_zones_jobs_3_sum_5_min_sov,st_density_units_90pct_plus,st_bg_mean_year_built,st_bg_prop_units_sf,st_bg_mean_age_of_head,st_bg_mean_children,st_bg_median_value_13_acs,st_bg_ratio_households_to_units,...,st_tracts_income_segment_1_hh,st_zones_total_jobs_sum_20_min_sov,st_zones_jobs_5_sum_5_min_sov,st_zones_jobs_1_sum_5_min_sov,st_zones_density_units_ave_5_min_sov,st_zones_income_6_hh_sum_10_min_sov,st_zones_jobs_0_sum_5_min_sov,st_pred_built_out_sf,st_zones_jobs_2_sum_5_min_sov,st_bg_prop_hh_rent
block_id,,,,,,,,,,,,,,,,,,,,,
060014271001001,949,06001,-0.744329,-0.333334,-2.195642,-0.541764,0.453005,-0.411325,0.051995,0.806891,...,-0.619544,-0.394193,-0.73433,-0.778006,0.884604,-0.732017,-0.515068,-0.125313,-0.713458,0.422013
060014271001002,949,06001,-0.744329,-0.333334,-2.195642,-0.541764,0.453005,-0.411325,0.051995,0.806891,...,-0.619544,-0.394193,-0.73433,-0.778006,0.884604,-0.732017,-0.515068,-0.125313,-0.713458,0.422013
060014271001003,949,06001,-0.744329,-0.333334,-2.195642,-0.541764,0.453005,-0.411325,0.051995,0.806891,...,-0.619544,-0.394193,-0.73433,-0.778006,0.884604,-0.732017,-0.515068,-0.125313,-0.713458,0.422013
060014271001004,949,06001,-0.744329,-0.333334,-2.195642,-0.541764,0.453005,-0.411325,0.051995,0.806891,...,-0.619544,-0.394193,-0.73433,-0.778006,0.884604,-0.732017,-0.515068,-0.125313,-0.713458,0.422013
060014271001005,949,06001,-0.744329,-0.333334,-2.195642,-0.541764,0.453005,-0.411325,0.051995,0.806891,...,-0.619544,-0.394193,-0.73433,-0.778006,0.884604,-0.732017,-0.515068,-0.125313,-0.713458,0.422013


In [27]:
model.chooser_filters = [
            filter for filter in model.out_chooser_filters.split('&') if 'block_id' not in filter]

In [28]:
choosers = get_data(
            model.choosers,
            filters=model.chooser_filters,
            model_expression=model.model_expression,
            extra_columns=[model.choice_column]
        )

In [49]:
mct = MergedChoiceTable(
            choosers,
            alternatives,
            chosen_alternatives=model.choice_column,
            sample_size=5
        )

In [50]:
a = mct.to_frame()

In [51]:
a.head(6)

lcm_county_id  hh_type zone_id county_id  \
household_id block_id                                                   
2891841      060014419244011         06001        1     778     06001   
             060014014002010         06001        1     978     06001   
             060014211002017         06001        1    1030     06001   
             060019820001025         06001        1     966     06001   
             060014506025009         06001        1     736     06001   
2891712      060014382012027         06001        1     807     06001   

                              st_zones_jobs_3_sum_5_min_sov  \
household_id block_id                                         
2891841      060014419244011                      -0.333945   
             060014014002010                       0.100465   
             060014211002017                      -0.312883   
             060019820001025                      -0.466149   
             060014506025009                       0.024862   
2891712      060014382012027                       0.079197   

                              st_density_units_90pct_plus  \
household_id block_id                                       
2891841      060014419244011                     2.999971   
             060014014002010                     2.999971   
             060014211002017                    -0.333334   
             060019820001025                    -0.333334   
             060014506025009                    -0.333334   
2891712      060014382012027                    -0.333334   

                              st_bg_mean_year_built  st_bg_prop_units_sf  \
household_id block_id                                                      
2891841      060014419244011               0.079541            -0.715395   
             060014014002010              -1.187863            -1.686313   
             060014211002017              -2.192941             1.004126   
             060019820001025               0.435259            -0.860379   
             060014506025009               0.679745            -0.186085   
2891712      060014382012027               0.194314            -2.157224   

                              st_bg_mean_age_of_head  st_bg_mean_children  \
household_id block_id                                                       
2891841      060014419244011               -0.355625            -0.438719   
             060014014002010               -0.475763             0.342485   
             060014211002017                1.863530            -0.404563   
             060019820001025               -2.354673            -0.577958   
             060014506025009                0.535196             1.276299   
2891712      060014382012027                0.700425             0.070167   

                              ...  st_zones_total_jobs_sum_20_min_sov  \
household_id block_id         ...                                       
2891841      060014419244011  ...                           -0.149949   
             060014014002010  ...                            0.470666   
             060014211002017  ...                           -0.542582   
             060019820001025  ...                           -0.110032   
             060014506025009  ...                           -0.223190   
2891712      060014382012027  ...                            0.107977   

                              st_zones_jobs_5_sum_5_min_sov  \
household_id block_id                                         
2891841      060014419244011                      -0.346713   
             060014014002010                       0.279513   
             060014211002017                      -0.393476   
             060019820001025                      -0.475662   
             060014506025009                      -0.025157   
2891712      060014382012027                      -0.006756   

                              st_zones_jobs_1_sum_5_min_sov  \
household_id block_id                                         
2891841 

In [52]:
model.mct_intx_ops = mct_intx_ops
new_mct = model.perform_mct_intx_ops(mct)

Replacing MCT None's and NaN's with 0


In [55]:
new_mct.to_frame().head(10)

lcm_county_id  hh_type zone_id county_id  \
household_id block_id                                                   
2891841      060014419244011         06001        1     778     06001   
             060014014002010         06001        1     978     06001   
             060014211002017         06001        1    1030     06001   
             060019820001025         06001        1     966     06001   
             060014506025009         06001        1     736     06001   
2891712      060014382012027         06001        1     807     06001   
             060014283021003         06001        1     953     06001   
             060014090004016         06001        1     874     06001   
             060014001001002         06001        1    1155     06001   
             060014403341017         06001        1     796     06001   

                              st_zones_jobs_3_sum_5_min_sov  \
household_id block_id                                         
2891841      060014419244011                      -0.333945   
             060014014002010                       0.100465   
             060014211002017                      -0.312883   
             060019820001025                      -0.466149   
             060014506025009                       0.024862   
2891712      060014382012027                       0.079197   
             060014283021003                      -0.249966   
             060014090004016                      -0.029624   
             060014001001002                      -0.500706   
             060014403341017                       1.350770   

                              st_density_units_90pct_plus  \
household_id block_id                                       
2891841      060014419244011                     2.999971   
             060014014002010                     2.999971   
             060014211002017                    -0.333334   
             060019820001025                    -0.333334   
             060014506025009                    -0.333334   
2891712      060014382012027                    -0.333334   
             060014283021003                    -0.333334   
             060014090004016                    -0.333334   
             060014001001002                    -0.333334   
             060014403341017                    -0.333334   

                              st_bg_mean_year_built  st_bg_prop_units_sf  \
household_id block_id                                                      
2891841      060014419244011               0.079541            -0.715395   
             060014014002010              -1.187863            -1.686313   
             060014211002017              -2.192941             1.004126   
             060019820001025               0.435259            -0.860379   
             060014506025009               0.679745            -0.186085   
2891712      060014382012027               0.194314            -2.157224   
             060014283021003               0.566628             1.048905   
             060014090004016               2.154352             0.955299   
             060014001001002               1.755558             1.068955   
             060014403341017               1.009490             0.908818   

                              st_bg_mean_age_of_head  st_bg_mean_children  \
household_id block_id                                                       
2891841      060014419244011               -0.355625            -0.438719   
             060014014002010               -0.475763             0.342485   
             060014211002017                1.863530            -0.404563   
             060019820001025               -2.354673            -0.577958   
             060014506025009                0.535196             1.276299   
2891712      060014382012027                0.700425             0.070167   
             060014283021003                0.844479             0.518931   
             060014090004016               -5.989717    

In [56]:
model.fit(new_mct)

PatsyError: Error evaluating factor: NameError: name 'mandatory_sov' is not defined
    mandatory_activity_dummy:mandatory_sov + st_bg_mean_age_of_head + st_bg_mean_children + st_bg_mean_year_built + st_bg_median_value_13_acs + st_bg_prop_hh_rent + st_bg_prop_units_sf + st_bg_ratio_households_to_units + st_bg_total_units + st_density_jobs_90pct_plus + st_density_units_90pct_plus + st_pred_built_out_sf + st_pred_rich_owned_antique + st_tracts_income_segment_1_hh + st_tracts_income_segment_6_hh + st_zones_density_units_ave_5_min_sov + st_zones_income_6_hh_sum_10_min_sov + st_zones_jobs_0_sum_5_min_sov + st_zones_jobs_1_sum_5_min_sov + st_zones_jobs_2_sum_5_min_sov + st_zones_jobs_3_sum_5_min_sov + st_zones_jobs_4_sum_5_min_sov + st_zones_jobs_5_sum_5_min_sov + st_zones_total_jobs_sum_20_min_sov + st_zones_total_units_sum_10_min_sov - 1
                             ^^^^^^^^^^^^^

In [ ]:
orca.list_injectables()

In [ ]:
'skims' in orca.list_injectables()

In [ ]:
orca.get_injectable('calibrated_folder')

In [ ]:
region_code = orca.get_injectable("region_code")
calibrated_folder = orca.get_injectable("calibrated_folder")
skim_source = orca.get_injectable("skim_source")
calibrated_path = os.path.join(
    'configs', 'calibrated_configs/', calibrated_folder, region_code)

In [ ]:
calibrated_path

In [ ]:
skim_source